# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "cities.csv"
city_data_df = pd.read_csv(output_data_file)
city_data_df.head()



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ushuaia,40,AR,1584017313,93,-54.80,-68.30,282.15,2.47
1,Remontnoye,72,RU,1584017461,62,46.56,43.65,287.04,12.07
2,Noboribetsu,75,JP,1584017462,59,42.45,141.17,276.15,4.10
3,Vaini,90,TO,1584017464,94,-21.20,-175.20,298.15,1.00
4,Provideniya,100,RU,1584017465,98,64.38,-173.30,269.59,5.21


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with api key
gmaps.configure(api_key=g_key)

#Define locations as Lat and Lng
locations = city_data_df[["Lat", "Lng"]].astype(float)
weight = city_data_df["Humidity"].astype(float)


#Add heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Define weather conditions for vacation destination
vacay_df = city_data_df
vacay_df = vacay_df[(vacay_df["Max Temp"] > 296) & (vacay_df["Max Temp"] <=302)]
vacay_df = vacay_df[(vacay_df["Wind Speed"]) <= 7]
vacay_df = vacay_df[(vacay_df["Humidity"] > 30) & (vacay_df["Humidity"] <=60)]
vacay_df = vacay_df[(vacay_df["Cloudiness"]) <= 20]

#Drop null values and print dataframe
vacay_df.dropna()
vacay_df
 

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
87,Lashio,6,MM,1584017558,32,22.93,97.75,297.19,1.59
206,Thaton,0,MM,1584017692,38,16.92,97.37,299.99,2.78
216,São Filipe,0,CV,1584017703,55,14.90,-24.50,298.19,6.58
242,Surat,0,IN,1584017464,36,21.17,72.83,299.15,2.60
291,Walvis Bay,3,NaN,1584017786,44,-22.96,14.51,299.15,4.60
457,Verāval,0,IN,1584017970,38,20.90,70.37,298.09,6.58
470,Tura,0,IN,1584017985,38,25.52,90.22,300.09,1.87
484,Mīnāb,0,IR,1584018000,47,27.15,57.08,299.15,5.10
540,Oranjemund,0,NaN,1584018068,52,-28.55,16.43,296.19,5.45


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Define hotel_df and add Hotel Name as a column to the dataframe
hotel_df = vacay_df
hotel_df.insert(9, "Hotel Name", " ")
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
87,Lashio,6,MM,1584017558,32,22.93,97.75,297.19,1.59,
206,Thaton,0,MM,1584017692,38,16.92,97.37,299.99,2.78,
216,São Filipe,0,CV,1584017703,55,14.90,-24.50,298.19,6.58,
242,Surat,0,IN,1584017464,36,21.17,72.83,299.15,2.60,
291,Walvis Bay,3,NaN,1584017786,44,-22.96,14.51,299.15,4.60,
457,Verāval,0,IN,1584017970,38,20.90,70.37,298.09,6.58,
470,Tura,0,IN,1584017985,38,25.52,90.22,300.09,1.87,
484,Mīnāb,0,IR,1584018000,47,27.15,57.08,299.15,5.10,
540,Oranjemund,0,NaN,1584018068,52,-28.55,16.43,296.19,5.45,


In [12]:
#Define variables for Google Place API parameters
target_radius = 5000
target_search_type = "lodging"

#Set up Parameters for Google Place API
params = {
    "radius": target_radius,
    "types": target_search_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # retrieve lat and lng values from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # Update location each iteration by using lat and lng values
    params["location"] = f"{lat},{lng}"
    
    #Define url 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Use url and params to make API call to retrieve data
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    #Retrieve results returned from API call
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        





Retrieving Results for Index 87: Lashio.
Closest hotel is Hotel CF Lashio - Burmese Only.
Retrieving Results for Index 206: Thaton.
Closest hotel is Blue Cloud.
Retrieving Results for Index 216: São Filipe.
Closest hotel is Tortuga B&B.
Retrieving Results for Index 242: Surat.
Closest hotel is Budget Inn Bellevue.
Retrieving Results for Index 291: Walvis Bay.
Closest hotel is Oyster Box Guesthouse.
Retrieving Results for Index 457: Verāval.
Closest hotel is Lords Inn Somnath.
Retrieving Results for Index 470: Tura.
Closest hotel is Hotel RIKMAN Continental.
Retrieving Results for Index 484: Mīnāb.
Closest hotel is فروشگاه موبایل ترکمانی.
Retrieving Results for Index 540: Oranjemund.
Closest hotel is Fisherman's Cove.


In [14]:

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
87,Lashio,6,MM,1584017558,32,22.93,97.75,297.19,1.59,Hotel CF Lashio - Burmese Only
206,Thaton,0,MM,1584017692,38,16.92,97.37,299.99,2.78,Blue Cloud
216,São Filipe,0,CV,1584017703,55,14.90,-24.50,298.19,6.58,Tortuga B&B
242,Surat,0,IN,1584017464,36,21.17,72.83,299.15,2.60,Budget Inn Bellevue
291,Walvis Bay,3,NaN,1584017786,44,-22.96,14.51,299.15,4.60,Oyster Box Guesthouse
457,Verāval,0,IN,1584017970,38,20.90,70.37,298.09,6.58,Lords Inn Somnath
470,Tura,0,IN,1584017985,38,25.52,90.22,300.09,1.87,Hotel RIKMAN Continental
484,Mīnāb,0,IR,1584018000,47,27.15,57.08,299.15,5.10,فروشگاه موبایل ترکمانی
540,Oranjemund,0,NaN,1584018068,52,-28.55,16.43,296.19,5.45,Fisherman's Cove


In [16]:
narrowed_city_df = hotel_df

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

#Display Map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig



Figure(layout=FigureLayout(height='420px'))